In [ ]:
#pip install avro

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import avro # should not raise ImportError

# Defining a schema

Les schémas Avro (façon de décrire la structure des données) sont définis en utilisant le format JSON.
Ce schéma définit un enregistrement représentant un utilisateur imaginaire (**un fichier = un unique schéma/une unique fiche**).
Ce qu'on est obligé de dire dans la fiche : 
1. namespace = nom de dossier
2. dire que c'est une fiche avec plusieurs infos = record --> c'est un objet avec plusieurs champs (formulaire)
3. lui donner un nom = User 
4. lister les infos qu'elle contient (name, favorite number, favorite color) --> chaque info est appelé champ

In [2]:
{"namespace": "example.avro",
 "type": "record",
 "name": "User",
 "fields": [
     {"name": "name", "type": "string"},
     {"name": "favorite_number",  "type": ["int", "null"]},
     {"name": "favorite_color", "type": ["string", "null"]}
 ]
}

{'namespace': 'example.avro',
 'type': 'record',
 'name': 'User',
 'fields': [{'name': 'name', 'type': 'string'},
  {'name': 'favorite_number', 'type': ['int', 'null']},
  {'name': 'favorite_color', 'type': ['string', 'null']}]}

# Serializing and deserializing without code generation


Avro stocke toujours le schéma avec les données. Ça veut dire : quand tu enregistres des infos, Avro garde aussi “la fiche qui dit à quoi ressemblent ces infos”. Donc même si tu n’as jamais vu le schéma avant, tu peux lire les données correctement.

**Sérialisation** = écrire les données dans un format Avro --> C’est comme mettre les infos dans une boîte spéciale qui garde aussi la notice pour les lire.
**Désérialisation** = lire les données depuis ce format --> C’est comme ouvrir la boîte et utiliser la notice pour comprendre les infos.

Exemple concret code Python qui :
-Met deux utilisateurs dans un fichier Avro (sérialisation)
-Lit ce fichier et récupère les utilisateurs (désérialisation)

In [4]:
import avro.schema #manipuler les schémas Avro
from avro.datafile import DataFileReader, DataFileWriter #lire et écrire des fichiers Avro
from avro.io import DatumReader, DatumWriter #transformer les données depuis/vers Avro

schema = avro.schema.parse(open("user.avsc", "rb").read()) #On ouvre le fichier user.avsc, qui contient le schéma de notre “fiche utilisateur”. | conversion en objet Python grâce à avro.schema.parse | schema contient maintenant la description de la fiche utilisateur (nom, favorite_number, favorite_color…).

writer = DataFileWriter(open("users.avro", "wb"), DatumWriter(), schema) #On crée un DataFileWriter, qui va nous permettre d’écrire des données dans un fichier Avro. | Le fichier s’appelle users.avro, on l’ouvre en mode écriture binaire (wb). | On utilise un DatumWriter pour transformer les données Python en format Avro. | On lui passe aussi le schéma que l’on a créé précédemment.
writer.append({"name": "Alyssa", "favorite_number": 256}) #On ajoute une fiche utilisateur au fichier Avro. 
writer.append({"name": "Ben", "favorite_number": 7, "favorite_color": "red"}) #On ajoute une autre fiche utilisateur.
writer.close() #On ferme le fichier. C’est important de toujours fermer les fichiers après les avoir utilisés, pour s’assurer que toutes les données sont bien écrites sur le disque.

reader = DataFileReader(open("users.avro", "rb"), DatumReader()) #On crée un DataFileReader, qui va nous permettre de lire les données depuis le fichier Avro. | On ouvre le fichier users.avro en mode lecture binaire (rb). | On utilise un DatumReader pour transformer les données Avro en objets Python.
for user in reader: #On itère sur chaque fiche utilisateur dans le fichier Avro.
    print(user)
reader.close() #On ferme le fichier après la lecture.

{'name': 'Alyssa', 'favorite_number': 256, 'favorite_color': None}
{'name': 'Ben', 'favorite_number': 7, 'favorite_color': 'red'}


**Assurez-vous d’ouvrir vos fichiers en mode binaire (c’est-à-dire en utilisant les modes wb ou rb respectivement)**

In [ ]:
schema = avro.schema.parse(open("user.avsc", "rb").read()) #

On veut lire le schéma de notre utilisateur (user.avsc) et le transformer en objet Python utilisable pour écrire et lire les fichiers Avro. Comment ça marche :

**open("user.avsc", "rb").read()** → lit tout le contenu du fichier en binaire.
**avro.schema.parse(...)** → prend ce texte JSON et le convertit en objet Python qui représente le schéma.

Résultat : schema n’est plus du texte, c’est un objet Avro qu’on peut utiliser pour :
-créer un fichier Avro (DataFileWriter)
-lire un fichier Avro (DataFileReader)

Dans ce cas précis, c’est un RecordSchema, car notre schéma décrit un record / une fiche avec plusieurs champs.

In [6]:
writer = DataFileWriter(open("users.avro", "wb"), DatumWriter(), schema) #On crée un “écrivain de fichier Avro” pour mettre nos utilisateurs dans un fichier sur le disque (pareil que précédemment).

In [10]:
writer.append({"name": "Alyssa", "favorite_number": 256})
writer.append({"name": "Ben", "favorite_number": 7, "favorite_color": "red"})
writer.close() 

In [11]:
reader = DataFileReader(open("users.avro", "rb"), DatumReader())

In [12]:
for user in reader:
    print(user)

{'name': 'Alyssa', 'favorite_number': 256, 'favorite_color': None}
{'name': 'Ben', 'favorite_number': 7, 'favorite_color': 'red'}
{'name': 'Alyssa', 'favorite_number': 256, 'favorite_color': None}
{'name': 'Ben', 'favorite_number': 7, 'favorite_color': 'red'}


| Méthode                                                           | Source du schéma                                                                                 | Type de données lu                   | Conversion nécessaire                                                                                    | Résultat final                                          | Usage principal                                                                                                  |
| ----------------------------------------------------------------- | ------------------------------------------------------------------------------------------------ | ------------------------------------ | -------------------------------------------------------------------------------------------------------- | ------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------- |
| `schema = json.loads(reader.meta['avro.schema'].decode('utf-8'))` | Schéma **stocké dans les métadonnées** d’un fichier Avro existant (`reader.meta['avro.schema']`) | Bytes (depuis le fichier Avro)       | `decode('utf-8')` pour transformer en chaîne de caractères, puis `json.loads` pour créer un dictionnaire | **Dictionnaire Python** (`dict`) contenant le schéma    | Lire et comprendre la structure du fichier Avro, accéder aux champs, pas pour écrire de nouveaux fichiers Avro   |
| `schema = avro.schema.parse(open("user.avsc", "rb").read())`      | Schéma **dans un fichier `.avsc` externe**                                                       | Bytes (lu depuis le fichier `.avsc`) | `avro.schema.parse()` fait **automatiquement** la conversion bytes → string → objet Avro                 | **Objet Avro** (`avro.schema.RecordSchema` par exemple) | Écrire ou lire des fichiers Avro, vérifier que les données respectent le schéma, sérialisation / désérialisation |


In [ ]:
Colonnes séparées         zip() transforme      list comprehension prépare CSV
timestamp x y z           tuples ligne par ligne   listes ligne par ligne
100       0.1 0.0 1.0   →  (100,0.1,0.0,1.0)  →  [100,0.1,0.0,1.0]
200       0.2 -0.1 0.9  →  (200,0.2,-0.1,0.9) →  [200,0.2,-0.1,0.9]
300       0.3 0.1 1.1   →  (300,0.3,0.1,1.1)  →  [300,0.3,0.1,1.1]